In [8]:
import pandas as pd
import pickle
import time
import tqdm
import numpy as np
import pprint

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader


from models.transformer import Model
from utils import TrainDataset, collate_fn, my_loss, train_loop
from torch.utils.tensorboard import SummaryWriter


In [9]:
# load data
data_path = 'mathorcup_recom_listwise/data/'
train_df = pd.read_csv(data_path + 'train_data.csv', dtype=str, nrows=10000)
train_userID = set(train_df['userID'])
userID2idx = {_: i for i, _ in enumerate(train_userID)}
with open(data_path + 'contentID2idx.pickle', 'rb') as handle:
    contentID2idx = pickle.load(handle)
with open(data_path + 'contentTC2ID.pickle', 'rb') as handle:
    contentTC2ID = pickle.load(handle)
item_len = len(contentTC2ID)
assert item_len == 1485

In [10]:
model = Model(user_len=len(train_userID), item_len=1485)
model.load_state_dict(torch.load(
    'saves/epoch20000-20220113-074529'
))
model.eval()



Model(
  (userID_embeddings): Embedding(1121, 6)
  (itemID_embeddings): Embedding(1486, 6)
  (date2vec): Date2Vec(
    (fc1): Linear(in_features=6, out_features=16, bias=True)
    (fc2): Linear(in_features=6, out_features=16, bias=True)
    (d2): Dropout(p=0.3, inplace=False)
    (fc3): Linear(in_features=32, out_features=16, bias=True)
    (d3): Dropout(p=0.3, inplace=False)
    (fc4): Linear(in_features=16, out_features=6, bias=True)
    (fc5): Linear(in_features=6, out_features=6, bias=True)
  )
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=6, out_features=6, bias=True)
          )
          (linear1): Linear(in_features=6, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=6, bias=True)
          (norm

In [11]:
train_dataset = TrainDataset(train_df, userID2idx, contentID2idx)

In [12]:
train_dataloader = DataLoader(
    train_dataset, collate_fn=collate_fn, batch_size=1, shuffle=True
)


In [13]:
total_energy=train_dataset.max_sum_duration
total_energy

2323

In [14]:
for i, batch in enumerate(train_dataloader):
    userIdxes, dates, itemIdxes, durations, lengths = batch
    # lengths = lengths.to(device)

    tgt_mask = model.get_tgt_mask(size=itemIdxes.size(1))
    tgt_key_padding_mask = model.create_pad_mask(matrix=itemIdxes, pad_token=item_len)

    preds = model(
        userIdxes,
        dates,
        itemIdxes,
        tgt_mask=tgt_mask,
        tgt_key_padding_mask=tgt_key_padding_mask
    ).squeeze()
    print(preds * total_energy)  # (N, T)
    print(durations)

tensor([0.7173, 0.7173, 0.7173, 0.7173, 0.7173, 0.7173, 0.7173],
       grad_fn=<MulBackward0>)
tensor([[0, 0, 0, 0, 0, 0, 0]], dtype=torch.int32)
tensor([0.7173, 0.7173, 0.7173, 0.7173, 0.7173], grad_fn=<MulBackward0>)
tensor([[0, 0, 0, 0, 0]], dtype=torch.int32)
tensor([0.7173, 0.7173, 0.7173, 0.7173, 0.7173, 0.7173, 0.7173],
       grad_fn=<MulBackward0>)
tensor([[  0,   0,  10,   0, 152,  17, 133]], dtype=torch.int32)
tensor([0.7173, 0.7173, 0.7173, 0.7173, 0.7173, 0.7173, 0.7173],
       grad_fn=<MulBackward0>)
tensor([[0, 0, 0, 0, 0, 0, 0]], dtype=torch.int32)
tensor([0.7173, 0.7173, 0.7173, 0.7173, 0.7173, 0.7173, 0.7173, 0.7173, 0.7173,
        0.7173], grad_fn=<MulBackward0>)
tensor([[281,   0,   0,   0, 315,   0,   0,   0,   0,   0]], dtype=torch.int32)
tensor([0.7173, 0.7173, 0.7173, 0.7173, 0.7173, 0.7173, 0.7173],
       grad_fn=<MulBackward0>)
tensor([[0, 0, 0, 0, 0, 0, 0]], dtype=torch.int32)
tensor([0.7173, 0.7173, 0.7173, 0.7173], grad_fn=<MulBackward0>)
tensor([[0, 0,

KeyboardInterrupt: 